In [2]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [91]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_america01.jpg'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [111]:
### Hyper-parameters ###
layerSize = 3 #number of filters in a layer
layerNum = 6 #number of layers
filterSize = 10 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter

In [113]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [114]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[  4.35360957e-03   3.28794148e-03   1.96667873e-03 ...,
      3.19505066e-03  -2.67052802e-03  -3.32463812e-04]
   [  4.64384560e-03   3.29614166e-03  -5.68943433e-04 ...,
     -1.12995036e-04   4.20481909e-05  -1.55732609e-03]
   [ -1.04603751e-03   3.99766948e-03  -2.83261145e-03 ...,
      6.89649725e-04  -3.36106108e-03  -3.30492073e-03]
   ..., 
   [  4.81032450e-03   3.66489786e-03   2.75049565e-03 ...,
     -1.22764691e-03   2.53000289e-03  -4.14927497e-03]
   [  4.15761338e-03  -3.68939617e-03  -1.55905096e-03 ...,
     -3.07359312e-03   3.69530374e-03  -1.88393556e-03]
   [  1.56611399e-03   1.57483731e-03  -7.13653559e-04 ...,
     -2.35557460e-04   9.11465663e-04   8.75436432e-04]]

  [[ -3.63515080e-03   1.49593595e-03  -4.87213008e-03 ...,
     -2.24507030e-03   3.55712078e-03  -1.53734660e-03]
   [  1.98477429e-03   4.42425981e-03  -4.44568584e-03 ...,
      1.89248611e-03  -3.87214505e-03   1.40115716e-03]
   [  3.80165340e-05  -4.00499878e-03   1.08412177e-03 ...,
 

In [115]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x < 0] = leak
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

0.000274608142686


In [116]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x)) #Add up all cross-correlations (with ReLU)
    return new_img

In [122]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
                #print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
                #print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2]) 
                #Apply corresponding filter
                print(filterNum*num + layerSize*lay1+lay2)
                print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

0
[[-0.01        0.00273019  0.004733   ...,  0.00137052  0.00387909
   0.00335336]
 [-0.01       -0.01       -0.01       ...,  0.01053397  0.01419237
   0.01534618]
 [ 0.00053955  0.01080698  0.00798488 ...,  0.01594621  0.01990618
   0.01779977]
 ..., 
 [-0.01        0.00603782  0.00489881 ...,  0.03903557  0.03665631
   0.0268901 ]
 [ 0.00146427  0.00998387  0.00537324 ...,  0.01807482  0.01561219
   0.00640339]
 [-0.01       -0.01       -0.01       ..., -0.01        0.00051224 -0.01      ]]
1
[[ 0.00271408  0.00149537 -0.01       ..., -0.01       -0.01       -0.01      ]
 [ 0.0020161   0.00161215 -0.01       ..., -0.01       -0.01       -0.01      ]
 [-0.01       -0.01       -0.01       ..., -0.01       -0.01       -0.01      ]
 ..., 
 [ 0.00327051  0.00288959  0.00522571 ..., -0.01       -0.01       -0.01      ]
 [ 0.0028886   0.00083355  0.00155432 ..., -0.01       -0.01       -0.01      ]
 [-0.01       -0.01       -0.01       ..., -0.01       -0.01       -0.01      ]]
2
[[ 0.012

In [118]:
def maxPool(size, img):
    new_img = np.zeros((int(len(img)/size), int(len(img[0])/size)))
    pad_img = np.pad(img, [(0, size-len(img)%size),(0, size-len(img[0])%size)], "constant", constant_values=(0))
    for x in range(math.ceil(len(img)/size)-1):
        for y in range(math.ceil(len(img[0])/size)-1):
            new_img[x][y] = np.amax(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
            print(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
    return new_img
print(maxPool(3, out[2]))

ValueError: Unable to create correctly shaped tuple from [(0, 3), (0, 3)]

In [121]:
### Error + Back-propagation ###
error = 1-np.mean(out[len(out)-1])
print(error)
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
        for row in filters[filterNum*layer+x-1]:
            c=0
            for w in row:
                new_grad[x][r][c]=-np.multiply(error*10,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer-1]),deriv=True,arr=True),
                    w),
                    grad[x][r][c])))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[filterNum*layer+x-1][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

0.999797979798
6


IndexError: index 53 is out of bounds for axis 0 with size 6

In [28]:
for x in range(epochs):
    out = forward(arr1r)
    error = 1-np.mean(out[len(out)-1])
    print(error)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))

0
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02   6.96378465e-05]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02   1.56251783e-04]
 ..., 
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02   7.11332929e-04 ...,   8.13456859e-03
    1.19016409e-02   8.83677533e-03]
 [ -1.00000000e-02   1.27609283e-03   2.85923745e-03 ...,   1.45389254e-02
    1.74496187e-02   1.33995265e-02]]
1
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02   4.98374952e-06 ...,   7.96337160e-05
    7.96337160e-05   7.96337160e-05]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,   2.22822067e-05
    2.22822067e-0

In [27]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'warn', 'under': 'ignore'}